In [ ]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
import json

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
DF_info = pd.read_csv(f"{path_work}/train_nn/TropiGATv2.final_df_v2.filtered.tsv", sep = "\t" ,  header = 0)

DF_info_lvl_0 = DF_info.copy()
df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))

# *****************************************************************************
# Functions :

def make_ensemble_TropiGAT_attention(path_ensemble) :
	"""
	This function builds a dictionary with all the models that are part of the TropiGAT predictor
	Input : Path of the models
	Output : Dictionary , attention weights
	# Make a json file with the versions of the GNN corresponding to each KL types
	# Load it
	# Create the correct model instance (TropiGAT_small_module or TropiGAT_big_module)
	"""
	errors = []
	DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.filtered.tsv", sep = "\t" ,  header = 0)
	DF_info_lvl_0 = DF_info.copy()
	df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
	dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))
	dico_ensemble = {}
	for GNN_model in os.listdir(path_ensemble) :
		if GNN_model[-2:] == "pt" :
			KL_type = GNN_model.split(".")[0]
			try :
				if dico_prophage_count[KL_type] >= 125 :
					model = TropiGAT_models.TropiGAT_big_module_attention(hidden_channels = 1280 , heads = 1)
				else :
					model = TropiGAT_models.TropiGAT_small_module_attention(hidden_channels = 1280 , heads = 1)
				model.load_state_dict(torch.load(f"{path_ensemble}/{GNN_model}"))
				dico_ensemble[KL_type] = model
			except Exception as e :
				a = (KL_type , dico_prophage_count[KL_type], e)
				errors.append(a)

	return dico_ensemble , errors

@torch.no_grad()
def make_predictions(model, data):
	model.eval() 
	output, weigths = model(data)
	probabilities = torch.sigmoid(output)
	predictions = probabilities.round() 
	return predictions, probabilities , weigths


def run_prediction_attentive(dico_graph, dico_ensemble, KL_type) :
    dico_predictions = {}
    query_graph = dico_graph[KL_type]
    model = dico_ensemble[KL_type]
    prediction, probabilities, weights = make_predictions(model, query_graph)
    dico_predictions[KL_type] = {"probabilitites" : probabilities, "weights" : weights}
    return dico_predictions

# *****************************************************************************
# Load the Models :
path_ensemble = f"{path_work}/train_nn/ensemble_2812"


dico_models, errors = TropiGAT_functions.make_ensemble_TropiGAT_attention(path_ensemble)
# *****************************************************************************
# Make graphs :
graph_baseline , dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(DF_info_lvl_0)
graph_dico = {kltype : TropiGAT_graph.build_graph_masking(graph_baseline , dico_prophage_kltype_associated,DF_info_lvl_0, kltype, 0, 1, 0, 0)
             for kltype in DF_info_lvl_0["KL_type_LCA"].unique()}


attention_data = {}
for kltype in graph_dico : 
    out_dico = run_prediction_attentive(graph_dico , dico_models, kltype)
    attention_data.update(out_dico)


with open(f"{path_work}/attention_weights_dico.json", "w") as outfile :
    json.dump(attention_data , outfile)
